<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/LSTM_nikkei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM モデルを用いた時系列データの予測問題
https://www.koi.mashykom.com/lstm.html

株価のデータは以下のリンクよりダウンロードしておく --> **https**://finance.yahoo.com/quote/%5EN225/history?p=%5EN225

データをプロットしてみる。
（usecols=[4] ... 4列目を使用）

In [ ]:
import pandas
import matplotlib.pyplot as plt
dataframe = pandas.read_csv('N225.csv', usecols=[4], engine='python', skipfooter=1)
dataset = dataframe.values
dataset = dataset.astype('float32')
plt.plot(dataset)
plt.show()

データの数値を[0,1]の範囲に正規化する。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# fix random seed for reproducibility
np.random.seed(7)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# split into train and test sets
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

データセットの変換

X が与えられた時間 (t, t-1, t-2) における株価、Y が時間 (t + 1) における株価であるようなデータセットを作成する。

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, maxlen):
    dataX, dataY = [], []
    for i in range(len(dataset)-maxlen-1):
        a = dataset[i:(i+maxlen), 0]
        dataX.append(a)
        dataY.append(dataset[i + maxlen, 0])
    return np.array(dataX), np.array(dataY)

# reshape into X=t and Y=t+maxlen
maxlen = 3
trainX, trainY = create_dataset(train, maxlen)
testX, testY = create_dataset(test, maxlen)

print (trainX[:10,:])
print (trainY[:10])

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX[:10,:])

LSTMモデルを作成し、学習させる。

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, maxlen)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

訓練および予測結果を表示する。

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate RMSE( root mean squared error)
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[maxlen:len(trainPredict)+maxlen, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(maxlen*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset), color ="g", label = "row")
plt.plot(trainPredictPlot,color="b", label="trainpredict")
plt.plot(testPredictPlot,color="m", label="testpredict")

plt.legend()
plt.show()